In [10]:
pip install catboost

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.tree import  DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import f1_score

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
data_path="/content/drive/MyDrive/Colab_Notebooks/Customer_Churn_Prediction/Churn-Data.csv"
data=pd.read_csv(data_path)
print(len(data.columns))
data=data.drop(['cID','TotalCharges'],axis=1)
data['Charges_Month']=data['Charges_Month'].apply(lambda x: int(x))


21


In [47]:
x = data.drop(["Churn"], axis=1)
y = pd.DataFrame(data[["Churn"]])
y["Churn"] = y["Churn"].apply(lambda x: 1 if x == "Yes" else 0)
non_numeric_columns = x.select_dtypes(include=['object']).columns
x = pd.get_dummies(x, columns=non_numeric_columns)

In [48]:
len(x.columns)

44

In [49]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
#pca = PCA(n_components=10)
#x_pca = pca.fit_transform(x_scaled)

In [50]:
#y_pca=data[["Churn"]]
smt=SMOTETomek(random_state=42)
rus=RandomUnderSampler(random_state=42)
x_r,y_r=smt.fit_resample(x_scaled,y)

In [51]:
len(x_r)

7914

In [52]:
x_train,x_test,y_train,y_test=train_test_split(x_r,y_r,train_size=0.8)

In [53]:
from sklearn.svm import SVC
model1=SVC()
model1.fit(x_train,y_train)
predict=model1.predict(x_test)
print(accuracy_score(y_test,predict))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8111181301326595


In [54]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [55]:
xgb_model = XGBClassifier()
xgb_model.fit(x_train, y_train)

lgb_model = LGBMClassifier()
lgb_model.fit(x_train, y_train)

cat_model = CatBoostClassifier(silent=True)
cat_model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 3152, number of negative: 3179
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003391 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2426
[LightGBM] [Info] Number of data points in the train set: 6331, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497868 -> initscore=-0.008530
[LightGBM] [Info] Start training from score -0.008530


In [56]:
pred_xgb=xgb_model.predict(x_test)
pred_lgb=lgb_model.predict(x_test)
pred_cat=cat_model.predict(x_test)

In [57]:
pred=(predict+pred_cat+pred_lgb+pred_xgb)/4

In [58]:
def argmax(pred):
    l=[]
    for i in pred:
        if i>0.5:
            l.append(1)
        else:
            l.append(0)
    return l
pred=argmax(pred)

In [59]:
print("Accuracy : ",accuracy_score(y_test,pred))
print("F1 score : ",f1_score(y_test,pred))

Accuracy :  0.86481364497789
F1 score :  0.8672456575682381
